In [1]:
import pandas as pd

def preprocess(df, threshold = 0):
    n = df.shape[0]
    flags = [0 for i in range(n)]
    emojis = [{} for i in range(n)]
    reviews = ['' for i in range(n)]
    for i in range(n):
        start = []
        s = df.iloc[i, 1]
        for j in range(len(s)):
            if s[j] == "[":
                start.append(j)
            elif len(start) == 0:
                reviews[i] += s[j]
            if s[j] == "]" and len(start) > 0:
                flags[i] = 1
                emoji = s[start[-1]:(j + 1)]
                if emoji not in emojis[i]:
                    emojis[i][emoji] = 0
                emojis[i][emoji] += 1
                start.pop()
    df['has_emoji'] = flags
    df['emojis'] = emojis
    df['no_emoji_review'] = reviews

In [2]:
df1 = pd.read_csv("Data/weibo_senti_100k.csv")
df2 = pd.read_csv("Data/simplifyweibo_4_moods.csv")
df2['label'] = df2['label'].apply(lambda x : 1 if x == 0 else 0)
df = pd.concat([df1, df2])
df['review'] = df['review'].apply(lambda x : x.split("//")[0] if "//" in x else x)
preprocess(df)
df.head()

,label,review,has_emoji,emojis,no_emoji_review
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你],1,{'[爱你]': 3},﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...,1,{'[哈哈]': 1},@张晓鹏jonathan 土耳其的事要认真对待，否则直接开除。@丁丁看世界 很是细心，酒店都...
2,1,姑娘都羡慕你呢…还有招财猫高兴……,0,{},姑娘都羡慕你呢…还有招财猫高兴……
3,1,美~~~~~[爱你],1,{'[爱你]': 1},美~~~~~
4,1,梦想有多大，舞台就有多大![鼓掌],1,{'[鼓掌]': 1},梦想有多大，舞台就有多大!


In [3]:
df.to_csv('Data/processed_data.csv')